<a href="https://colab.research.google.com/github/sandeeproyy/IITG-ML-Internship/blob/main/faq_next_word_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Next‑Word Predictor (Word‑Level LSTM) — Clean Notebook

**What this does (in plain English):**
- Takes your FAQ text, cleans and tokenizes it.
- Builds input/output pairs so the model learns to guess the *next word*.
- Trains an LSTM language model (Keras/TensorFlow).
- Saves the model + tokenizer and provides simple generation utilities.
- Each cell below is labeled with *what it does* so you can follow along quickly.


## Cell 1 — Imports and Basic Config

In [1]:

# Purpose: Import all required libraries and set basic reproducibility options.

import os, json, time, random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Reproducibility (best effort)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(tf.__version__)


2.19.0


## Cell 2 — Load the FAQ corpus (your text)

In [2]:

# Purpose: Define your text corpus. You can replace this with external file I/O if needed.
faqs = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""

# Light normalization
text = faqs.lower()
print("Characters in corpus:", len(text))


Characters in corpus: 5128


## Cell 3 — Tokenize text and build (inputs, labels) for next‑word prediction

In [9]:

# Purpose: Convert text to tokens, build rolling n-gram sequences, pad them, and split into X (inputs) and y (labels).

tokenizer = Tokenizer(oov_token="<oov>")
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index
index_word = tokenizer.index_word  # reverse lookup
vocab_size = len(word_index) + 1   # +1 for padding token

print("Vocab size:", vocab_size)

# Convert the whole text to a single sequence of tokens
tokens = tokenizer.texts_to_sequences([text])[0]

# Build input sequences in a sliding window manner
# For seq_len N, we predict the (N+1)-th token.
seq_len = 15  # you can increase to give the model more context
sequences = []
for i in range(seq_len, len(tokens)):
    seq = tokens[i-seq_len:i+1]  # N tokens for input + 1 label
    sequences.append(seq)

sequences = np.array(sequences, dtype=np.int32)
X, y = sequences[:, :-1], sequences[:, -1]

# Pad inputs to a consistent length (should already be seq_len here, but padding keeps it robust)
max_len = X.shape[1]
X = pad_sequences(X, maxlen=max_len, padding="pre")

# One-hot encode labels for categorical crossentropy
y = to_categorical(y, num_classes=vocab_size)

print("X shape:", X.shape, "y shape:", y.shape, "seq_len:", max_len)


Vocab size: 284
X shape: (925, 15) y shape: (925, 284) seq_len: 15


## Cell 4 — Build the LSTM language model

In [10]:

# Purpose: Define an Embedding + stacked LSTM + Dense softmax model suitable for next-word prediction.

def build_model(vocab_size: int, seq_len: int):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128, input_length=seq_len),
        LSTM(150, return_sequences=True),  # return sequence to feed next LSTM
        Dropout(0.2),
        LSTM(150),
        Dropout(0.2),
        Dense(vocab_size, activation="softmax"),
    ])
    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    return model

model = build_model(vocab_size, max_len)
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Cell 5 — Train the model (with EarlyStopping and checkpoints)

In [11]:

# Purpose: Fit the model. On small text, reduce epochs to avoid overfitting.
os.makedirs("chkpts", exist_ok=True)

callbacks = [
    EarlyStopping(monitor="loss", patience=3, restore_best_weights=True),
    ModelCheckpoint("chkpts/faq_nextword_lstm.keras", monitor="loss", save_best_only=True)
]

history = model.fit(
    X, y,
    epochs=30,           # increase if you have more data
    batch_size=64,
    verbose=1,
    callbacks=callbacks,
    validation_split=0.1 # optional sanity check
)

# Save final model and tokenizer
model.save("faq_nextword_lstm_final.keras")
with open("faq_tokenizer.json", "w", encoding="utf-8") as f:
    f.write(tokenizer.to_json())

print("Artifacts saved: faq_nextword_lstm_final.keras, faq_tokenizer.json")


Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 139ms/step - accuracy: 0.0372 - loss: 5.6383 - val_accuracy: 0.0108 - val_loss: 5.6058
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.0805 - loss: 5.2507 - val_accuracy: 0.0108 - val_loss: 5.9000
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.0549 - loss: 5.0086 - val_accuracy: 0.0108 - val_loss: 6.0980
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.0809 - loss: 4.9707 - val_accuracy: 0.0108 - val_loss: 6.2830
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.0808 - loss: 4.9667 - val_accuracy: 0.0108 - val_loss: 6.3516
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.0825 - loss: 4.9460 - val_accuracy: 0.0108 - val_loss: 6.3813
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 184ms/step - accuracy: 0.0808 - loss: 4.9520 - val_accuracy: 0.0108 - val_loss: 6.3684
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 101ms/step - accuracy: 0.0829 - loss: 4.9225 - val_accuracy: 0.0108

## Cell 6 — Helper functions: temperature sampling and next‑word generation

In [12]:

# Purpose: Provide convenient utilities for generating text with adjustable creativity.

def sample_with_temperature(probs: np.ndarray, temperature: float = 1.0) -> int:
    probs = np.asarray(probs).astype("float64")
    if temperature <= 0:
        return int(np.argmax(probs))
    logits = np.log(np.maximum(probs, 1e-9)) / temperature
    exp = np.exp(logits)
    probs = exp / np.sum(exp)
    return int(np.random.choice(len(probs), p=probs))

def generate_next_words(seed_text: str, num_words: int = 10, temperature: float = 0.8):
    text_seq = seed_text.lower().split()
    for _ in range(num_words):
        # Encode last seq_len tokens
        encoded = tokenizer.texts_to_sequences([" ".join(text_seq)])[-1]
        encoded = encoded[-max_len:]  # keep only last max_len tokens
        padded = pad_sequences([encoded], maxlen=max_len, padding="pre")

        preds = model.predict(padded, verbose=0)[0]
        next_idx = sample_with_temperature(preds, temperature=temperature)

        # Convert index -> word (fallback to argmax if not found)
        word = index_word.get(next_idx)
        if (word is None) or (word == "<oov>"):
            next_idx = int(np.argmax(preds))
            word = index_word.get(next_idx, "")

        text_seq.append(word)
    return " ".join(text_seq)


## Cell 7 — Quick demo: generate text

In [13]:

# Purpose: Try a quick generation pass with greedy and temperature sampling.

seed = "what is the fee"
print("Seed:", seed)

print("\nGreedy (temperature=0):")
print(generate_next_words(seed, num_words=12, temperature=0.0))

print("\nSampling (temperature=0.8):")
print(generate_next_words(seed, num_words=12, temperature=0.8))


Seed: what is the fee

Greedy (temperature=0):
what is the fee the the i i i i i i i i i i

Sampling (temperature=0.8):
what is the fee time a a a can campusx and can yes i of can


## Cell 8 — Sanity checks & troubleshooting

In [14]:

# Purpose: Quick checks to debug shape or vocabulary issues if anything breaks.

print("Tokenizer vocab size:", vocab_size)
print("Sequence length used for training:", max_len)
print("X shape:", X.shape, "y shape:", y.shape)

# Check a few tokens and reverse lookups
sample_words = list(word_index.keys())[:10]
print("Sample tokens:", sample_words)
print("Indexes:", [word_index[w] for w in sample_words])
print("Reverse lookup check:", [index_word[i] for i in [1,2,3,4,5] if i in index_word])


Tokenizer vocab size: 284
Sequence length used for training: 15
X shape: (925, 15) y shape: (925, 284)
Sample tokens: ['<oov>', 'the', 'you', 'i', 'to', 'a', 'of', 'is', 'have', 'will']
Indexes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Reverse lookup check: ['<oov>', 'the', 'you', 'i', 'to']
